In [1]:
import string
import sys
import re
import os

In [18]:
filename = 'midi'
theText = open(filename, "r").read()
theTextLines = open(filename, "r").readlines()
startText = 'MIDI_IN  \* none'
endText = '        \* none \* 99'
skiptText = 'N\$*'

In [19]:
with open(filename) as myFile:
        for num, line in enumerate(myFile,1):
                if startText in line:
                    startLine = num - 1 #first line is off by 1 for some reason
                if endText in line:
                    endLine = num
        lineRange = range(startLine,endLine)

In [20]:
def changeToInt(s):
    try:
        int(s)
        return int(s)
    except ValueError:
        return s

In [22]:
pinDict = dict()
for i in lineRange:
    #print('------')
    print(theTextLines[i])
    thisline = re.split(" +", theTextLines[i])
    #print(thisline)
    #pinNum = re.sub('DAC0','A21',thisline[1])
    #pinNum = re.sub('DAC1','A22',pinNum)
    pinSig = re.sub('\n','',thisline[2])
    test = re.search(skiptText,theTextLines[i])
    #print(test)
    if test is None :
        pinDict[pinSig] = changeToInt(pinNum)
        #pinDict[pinSig] = pinNum

         * none * 49         io       N$68

         * none * 5          io       AN6

         * none * 50         io       N$69

         * none * 51         io       N$70

         * none * 52         io       N$71

         * none * 53         io       N$72

         * none * 54         io       N$73

         * none * 55         io       N$74

         * none * 56         io       N$75

         * none * 57         io       N$76

         * none * 58         io       N$77

         * none * 59         io       N$78

         * none * 6          io       AN7

         * none * 60         io       N$79

         * none * 61         io       N$80

         * none * 62         io       N$81

         * none * 63         io       N$82

         * none * 64         io       N$83

         * none * 65         io       N$84

         * none * 66         io       N$85

         * none * 67         io       N$86

         * none * 68         io       N$87

         * none * 69         io   

IndexError: list index out of range

In [303]:
pinDict

{'AN1': 14,
 'AN10': 'A22',
 'AN11': 'A10',
 'AN12': 'A11',
 'AN2': 15,
 'AN3': 16,
 'AN4': 33,
 'AN5': 34,
 'AN6': 35,
 'AN7': 36,
 'AN8': 39,
 'AN9': 'A21',
 'BTN1': 11,
 'BTN10': 44,
 'BTN11': 45,
 'BTN12': 46,
 'BTN2': 12,
 'BTN3': 24,
 'BTN4': 25,
 'BTN5': 26,
 'BTN6': 27,
 'BTN7': 28,
 'BTN8': 42,
 'BTN9': 43,
 'I2C_SCL': 47,
 'I2C_SDA': 48,
 'JOY_AN_H_M': 18,
 'JOY_AN_H_S': 20,
 'JOY_AN_V_M': 17,
 'JOY_AN_V_S': 19,
 'JOY_BTN_M': 0,
 'JOY_BTN_S': 1,
 'L1': 2,
 'L10': 30,
 'L11': 31,
 'L12': 32,
 'L2': 3,
 'L3': 4,
 'L4': 5,
 'L5': 6,
 'L6': 7,
 'L7': 8,
 'L8': 23,
 'L9': 29,
 'M_A_IN': 40,
 'M_B_IN': 41,
 'M_PWM_A': 21,
 'M_PWM_B': 22,
 'S1': 9,
 'S2': 10,
 'U1': 51,
 'U2': 52,
 'U3': 53,
 'U4': 54}

In [304]:
pots, joyPots, btns, joyBtns, leds, usr = ([] for i in range(6))
arrayNames = {'POTS' : pots,
              'JPOTS' : joyPots, 
              'BTNS' : btns,
              'JBTNS' : joyBtns,
              'LEDS' : leds, 
              'USR' : usr}

In [305]:
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    return [ atoi(c) for c in re.split('(\d+)', text) ]

In [306]:

def createList(searchTerm, listToAppendTo):
    listToAppendTo.clear()
    for key in pinDict.keys():
        if key.startswith(searchTerm):
            listToAppendTo.append(key)
            listToAppendTo.sort(key=natural_keys)
            
createList('BTN', btns)
createList('AN', pots)
createList('JOY_AN', joyPots)
createList('JOY_BTN', joyBtns)
createList('L',leds)
createList('U',usr)
leds += usr #combine leds and usr

In [308]:
output = open("output.h", "w")

#Create variables based on pinout
for key, value in pinDict.items():
    #outputLine = "int " + key + "\t" + "= " + str(value) + ";\n"
    outputLine = '{}{:10}{}{}{}'.format('const int ',key,'= ',value,';\n')
    output.write(outputLine)
 
#Create arrays for variables for looping 
output.write('\n')
output.write('//------------')
output.write('---ARRAYS---')
output.write('------------\n')
output.write('\n')
for key, value in arrayNames.items():
    
    outputLine = '{}{:10}{}'.format('const int ',key +'[]','= {')
    outputLine += '{}{}{}'.format(value[0],',','\n')
    for val in value[1 :-1]:
        outputLine += '{:>23}{}{}{}'.format(' ',val,',','\n')
    outputLine += '{:>23}{}{}{}'.format(' ',value[-1],'}',';\n')
    outputLine += '\n'
    output.write(outputLine)
  
output.close() 

In [309]:
print(arrayNames.items())

dict_items([('POTS', ['AN1', 'AN2', 'AN3', 'AN4', 'AN5', 'AN6', 'AN7', 'AN8', 'AN9', 'AN10', 'AN11', 'AN12']), ('JPOTS', ['JOY_AN_H_M', 'JOY_AN_H_S', 'JOY_AN_V_M', 'JOY_AN_V_S']), ('BTNS', ['BTN1', 'BTN2', 'BTN3', 'BTN4', 'BTN5', 'BTN6', 'BTN7', 'BTN8', 'BTN9', 'BTN10', 'BTN11', 'BTN12']), ('JBTNS', ['JOY_BTN_M', 'JOY_BTN_S']), ('LEDS', ['L1', 'L2', 'L3', 'L4', 'L5', 'L6', 'L7', 'L8', 'L9', 'L10', 'L11', 'L12', 'U1', 'U2', 'U3', 'U4']), ('USR', ['U1', 'U2', 'U3', 'U4'])])
